## Using neural networks to predict on Kryptonite-N dataset

In [1]:
import os
myseed = 6095 
os.environ['PYTHONHASHSEED'] = str(myseed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]= ":4096:8"
print(os.getenv("CUBLAS_WORKSPACE_CONFIG"))

import numpy as np
import random
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import product

:4096:8


In [2]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))


def set_seeds(myseed):

    random.seed(myseed)
    np.random.seed(myseed)
    torch.manual_seed(myseed)
    torch.cuda.manual_seed(myseed)
    torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)


set_seeds(myseed)


2.5.1+cu118
Device: cuda


In [3]:
import matplotlib.pyplot as plt


def plot_features_vs_label(X, y):

    n_samples, n_features = X.shape
    
    assert y.shape == (n_samples,)
    assert set(np.unique(y)).issubset({0, 1})
    
    for i in range(n_features):
        plt.figure()
        
        plt.scatter(X[y == 0, i], [0] * sum(y == 0), color='blue', label='Label 0', alpha=0.6)
        plt.scatter(X[y == 1, i], [1] * sum(y == 1), color='red', label='Label 1', alpha=0.6)
        
        plt.xlabel(f'Feature {i+1}')
        plt.ylabel('Label')
        plt.title(f'Feature {i+1} vs. Label')
        plt.legend()
        plt.grid(True)
        
        plt.show()


n = 15
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# plot_features_vs_label(X, y)


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=9, hidden_size=5, depth= 2):
        super().__init__()
        # self.linear_layer_stack = nn.Sequential(
        #     nn.Linear(input_size, hidden_size),
        #     nn.ReLU(),
        #     nn.Linear(hidden_size,hidden_size), 
        #     nn.ReLU(),
        #     nn.Linear(hidden_size,1), 
        # )



        modules = [torch.nn.Linear(input_size,hidden_size)]

        for i in range(depth-1):
            modules.append(torch.nn.Linear(hidden_size,hidden_size))
            modules.append(torch.nn.ReLU())

        modules.append(torch.nn.Linear(hidden_size, 1))

        self.linear_layer_stack = torch.nn.Sequential(
            *modules,
        )
 
        self._initialize_weights()  

    def _initialize_weights(self):
        for layer in self.linear_layer_stack:
            if isinstance(layer, nn.Linear):
                torch.nn.init.xavier_uniform_(layer.weight)
                torch.nn.init.zeros_(layer.bias)


    def forward(self, x):
        return self.linear_layer_stack(x)

In [5]:



# train_size = len(X_train)

def train_nn(model, dataloader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.unsqueeze(1).to(device)  # Move data to device
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}')

        # if epoch%10 == 0:
        #     validate_nn(model, )



#### Validation

In [6]:
def validate_nn(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val.to(device))
        val_outputs = torch.round(torch.sigmoid(val_outputs)).cpu().numpy()
        accuracy = accuracy_score(y_val, val_outputs)
    return accuracy

#### Grid Search Hyperparameter Tuning

In [7]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


In [8]:
def grid_search(X_train, y_train, X_val, y_val, param_grid, krypto_n):
    best_accuracy = 0
    best_params = None
    best_model = None
    
    for depth, hidden_size, learning_rate, batch_size, epochs in product(*param_grid.values()):


        print(f"Training with depth={depth}, hidden_size={hidden_size}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=depth).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


        g = torch.Generator()
        g.manual_seed(myseed)
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
        
        train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
        
        accuracy = validate_nn(model, X_val, y_val)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'depth':depth, 'hidden_size': hidden_size, 'learning_rate': learning_rate, 
                           'batch_size': batch_size, 'epochs': epochs}
            best_model = model

        print(f"Current best accuracy: {best_accuracy:.4f}")

    
    print("Best Parameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print(f"Krypto variant: {krypto_n}")
    
    
    return best_model, best_params

#### Random Grid Search

In [9]:
def rand_grid_search(X_train, y_train, X_val, y_val, krypto_n):
    best_accuracy = 0
    best_params = None
    best_model = None
    
    combos = 0
    max_combos = 10

    while best_accuracy < 0.90:


        # hidden_sizes = param_grid['hidden_size']
        # learning_rates = param_grid~learning_rate']
        # batch_sizes = param_grid['batch_size']
        # epoch_list = param_grid['epochs']

        hidden_size = np.random.randint(10,4000)
        learning_rate = np.random.uniform(0.0003, 0.0045)
        batch_size = np.random.randint(64,6000)
        epochs = np.random.randint(15, 50)
        depth = np.random.randint(2,3)



        print(f"Training with depth={depth}, hidden_size={hidden_size}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size, depth=depth).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        g = torch.Generator()
        g.manual_seed(myseed)
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
        
        train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
        
        accuracy = validate_nn(model, X_val, y_val)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'depth': [depth], 'hidden_size': [hidden_size], 'learning_rate': [learning_rate], 
                           'batch_size': [batch_size], 'epochs': [epochs]}
            best_model = model

        print(f"Current best accuracy: {best_accuracy:.4f}")
        combos += 1


    print("Best Parameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print(f"Krypto variant: {krypto_n}")
    print(combos)
    return best_model, best_params

In [10]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"
print('Device: {0}'.format(device))

# setting kryptonite dataset no.
n = 24

# Reading and normalising dataset features for efficient convergence
scaler = StandardScaler()

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# print(f"X shape: {X.shape()}")
print(np.shape(X))

if n>18:
    X_add = np.load('Datasets/additional-kryptonite-%s-X.npy'%(n))
    print(np.shape(X_add))
    y_add = np.load('Datasets/additional-kryptonite-%s-y.npy'%(n))

    X=  np.concatenate((X,X_add),axis = 0)
    y=  np.concatenate((y,y_add),axis = 0)


print(np.shape(X))


# print((X[0]))

# Shuffle and split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=myseed)  # 60% training

scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_temp)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=myseed)  # 20% validation, 20% test




X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)


print((X_train[0]))
print((X_val[0]))
print((X_test[0]))





2.5.1+cu118
Device: cuda
(48000, 24)
(250000, 24)
(298000, 24)
tensor([-0.0065,  0.9878,  0.0102, -0.2089,  0.9964,  0.9076,  1.0036, -0.0173,
        -0.0193,  1.0783, -0.0974,  0.9974,  0.9904,  0.9387, -0.0016, -0.0183,
         1.2517,  0.9654,  1.0247,  1.1510,  0.9289,  0.0350,  0.0277,  0.0029])
tensor([ 1.0975, -0.1971,  0.9907,  0.0894,  0.9935, -0.0323,  0.9932,  0.0048,
         0.1175,  0.1451,  0.0013,  0.0101, -0.0165,  0.0227,  0.9879,  1.0080,
         0.0529,  0.9902, -0.0065, -0.0077, -0.0546,  0.1071, -0.0777,  0.9921])
tensor([-0.0943, -0.1331,  0.0378, -0.0773,  0.0017,  0.9457, -0.0123,  0.8646,
         0.8112,  0.8649,  0.0140,  1.0167, -0.0204, -0.0206,  1.0240,  1.0195,
         1.0058, -0.0150,  0.0145,  1.0531, -0.0390,  0.1013,  0.9841, -0.0077])


In [11]:

# param_grid = {
#     'depth': [2,4,8,16],
#     'hidden_size': [8, 16, 24, 30], 
#     'learning_rate': [0.001, 0.005, 0.01, 0.05 , 0.1], 
#     'batch_size': [16, 32, 64, 96],     
#     'epochs': [5, 10, 20, 30]              
# }

# Dictionary of hyperparameter values to search through

param_grid =  {'depth': [2], 'hidden_size': [1295], 'learning_rate': [0.002355056560959035], 'batch_size': [2469], 'epochs': [38]}


# print("Random Search")
# best_model, best_params = rand_grid_search(X_train, y_train, X_val, y_val, n)

set_seeds(myseed)

print("Reproducibility check")
best_model, best_params = grid_search(X_train, y_train, X_val, y_val, param_grid, n)


# Current Hyperparameters for each Kryptonite Variant:

# Best Parameters: {'depth': 3, 'hidden_size': 38, 'learning_rate': 0.00141, 'batch_size': 140, 'epochs': 35}
# Best Validation Accuracy: 0.9527777777777777
# Best Validation Accuracy: 0.955
# Best Validation Accuracy: 0.9546296296296296
# Best Validation Accuracy: 0.9512962962962963
# Best Validation Accuracy: 0.9555555555555556
# Best Validation Accuracy: 0.9535185185185185




# Test Accuracy = 0.9566666666666667

# Krypto variant: 9


# Best Parameters: {'depth': 2, 'hidden_size': 84, 'learning_rate': 0.002226459276380165, 'batch_size': 155, 'epochs': 146}
# Best Validation Accuracy: 0.9452777777777778
# Test Accuracy = 0.9497222222222222
# Krypto variant: 12


# Best Parameters: {'depth': 2, 'hidden_size': 54, 'learning_rate': 0.002077090842646019, 'batch_size': 147, 'epochs': 165}
# Best Validation Accuracy: 0.9342222222222222
# Test Accuracy = 0.9304444444444444
# Krypto variant: 15

# Current best accuracy: 0.9048
# Best Parameters: {'depth': 2, 'hidden_size': 97, 'learning_rate': 0.002937306209994177, 'batch_size': 100, 'epochs': 72}
# Best Validation Accuracy: 0.9048148148148148
# Test Accuracy = 0.9057407407407407
# Krypto variant: 18

# Training with depth=2, hidden_size=84, learning_rate=0.0008952525037794653, batch_size=155, epochs=146
# Validation Accuracy: 0.9427
# Krypto variant: 15

# Best Parameters: {'depth': 2, 'hidden_size': 150, 'learning_rate': 0.0007625120811532073, 'batch_size': 443, 'epochs': 45}
# Best Validation Accuracy: 0.86016
# Test Accuracy = 0.8596933333333333
# Krypto variant: 24

# Best Parameters: {'depth': 2, 'hidden_size': 388, 'learning_rate': 0.000522646472592812, 'batch_size': 485, 'epochs': 156}
# Best Validation Accuracy: 0.9032266666666666
# Best Validation Accuracy: 0.8842933333333334
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.8572266666666667
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.81512
# Best Validation Accuracy: 0.9123066666666667
# Best Validation Accuracy: 0.84404


# Krypto variant: 24

# NEW DATA:
# Training with depth=2, hidden_size=685, learning_rate=0.0026145410706330304, batch_size=4500, epochs=51
# Validation Accuracy: 0.8118



# Best Validation Accuracy: 0.5041834451901566
# Best Validation Accuracy: 0.7748322147651007
# Best Validation Accuracy: 0.8106263982102908
# Best Validation Accuracy: 0.7934451901565995
# Best Validation Accuracy: 0.8178635346756152
# Best Validation Accuracy: 0.7923266219239373
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
# Best Validation Accuracy: 0.7840268456375838
 
# Training with depth=2, hidden_size=1805, learning_rate=0.0026249175734212675, batch_size=4445, epochs=44

# 0.8185


# Best Validation Accuracy: 0.7513758389261745
# Best Validation Accuracy: 0.7513758389261745



# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.7789597315436242

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.8215883668903803

# Best Parameters: {'depth': 2, 'hidden_size': 951, 'learning_rate': 0.0018208178202594196, 'batch_size': 2989, 'epochs': 30}
# Best Validation Accuracy: 0.789317673378076
 



# Training with depth=2, hidden_size=1295, learning_rate=0.002355056560959035, batch_size=2469, epochs=38
# Current best accuracy: 0.8386


Reproducibility check
Training with depth=2, hidden_size=1295, learning_rate=0.002355056560959035, batch_size=2469, epochs=38
Epoch 1/38, Loss: 0.7222
Epoch 2/38, Loss: 0.6931
Epoch 3/38, Loss: 0.6925
Epoch 4/38, Loss: 0.6921
Epoch 5/38, Loss: 0.6918
Epoch 6/38, Loss: 0.6912
Epoch 7/38, Loss: 0.6906
Epoch 8/38, Loss: 0.6898
Epoch 9/38, Loss: 0.6894
Epoch 10/38, Loss: 0.6885
Epoch 11/38, Loss: 0.6881
Epoch 12/38, Loss: 0.6868
Epoch 13/38, Loss: 0.6859
Epoch 14/38, Loss: 0.6852
Epoch 15/38, Loss: 0.6843
Epoch 16/38, Loss: 0.6834
Epoch 17/38, Loss: 0.6826
Epoch 18/38, Loss: 0.6802
Epoch 19/38, Loss: 0.6794
Epoch 20/38, Loss: 0.6776
Epoch 21/38, Loss: 0.6779
Epoch 22/38, Loss: 0.6759
Epoch 23/38, Loss: 0.6737
Epoch 24/38, Loss: 0.6724
Epoch 25/38, Loss: 0.6711
Epoch 26/38, Loss: 0.6686
Epoch 27/38, Loss: 0.6675
Epoch 28/38, Loss: 0.6652
Epoch 29/38, Loss: 0.6620
Epoch 30/38, Loss: 0.6601
Epoch 31/38, Loss: 0.6575
Epoch 32/38, Loss: 0.6554
Epoch 33/38, Loss: 0.6536
Epoch 34/38, Loss: 0.6515

In [13]:
def test_nn(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_outputs = torch.round(torch.sigmoid(test_outputs)).cpu().numpy()
        accuracy = accuracy_score(y_test, test_outputs)
    return accuracy


test_accuracy = test_nn(best_model, X_test, y_test)
print(f"Test Accuracy = {test_accuracy}")

Test Accuracy = 0.5
